<a href="https://colab.research.google.com/github/efemehmetkarabulut/AYRIK-SISTEMLER-ILERI-OLASILIK/blob/main/Untitled4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Adım 1: Veri setini indiren fonksiyon
def download_dataset():
    file_path = '‪/content/bbc_data_1_2.xlsx'
    # Veri setini Kaggle'dan indirme veya farklı bir kaynaktan yükleme işlemi gerçekleştirilir
    # Burada veri setinin Kaggle üzerinden indirilmiş olduğunu varsayalım
    # Daha sonra veri seti dosyasını okuyarak bir DataFrame'e dönüştürülür
    data = pd.read_excel('bbc_data_1_2.xlsx')
    return data

# Adım 2: Veriyi temizleyen fonksiyon
def clean_text(text):
    # Noktalama işaretlerini kaldırma
    text = re.sub(r'[^\w\s]', '', text)
    # Birden fazla boşluğu tek boşlukla değiştirme
    text = re.sub(r'\s+', ' ', text)
    # Metni küçük harfe dönüştürme
    text = text.lower()
    return text

# Adım 3: Eksik değerleri doldurma
def fill_missing_values(data):
    data.dropna(inplace=True)
    return data

# Adım 4: Hiperparametre optimizasyonunu gerçekleştiren fonksiyon
def optimize_hyperparameters(X_train, y_train):
    # TF-IDF vektörlerini oluşturmak için bir vektörleme nesnesi oluşturulur
    tfidf_vectorizer = TfidfVectorizer()
    # Naive Bayes sınıflandırıcı modeli
    nb_classifier = MultinomialNB()
    # Pipeline oluşturulur
    nb_model = Pipeline([('tfidf', tfidf_vectorizer), ('clf', nb_classifier)])

    # Hiperparametre aralıkları
    param_grid = {'tfidf__max_df': [0.5, 0.75, 1.0],
                  'tfidf__ngram_range': [(1, 1), (1, 2)],
                  'clf__alpha': [0.1, 0.5, 1.0]}

    # GridSearchCV kullanarak hiperparametre optimizasyonu yapılır
    grid_search = GridSearchCV(nb_model, param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    return best_params

# Adım 5: Veriyi ayrıştıran fonksiyon
def split_data(data):
    X = data['text']
    y = data['category']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Adım 6: Sınıflandırma modelini eğiten fonksiyon
def train_model(X_train, y_train, best_params):
    tfidf_vectorizer = TfidfVectorizer(max_df=best_params['tfidf__max_df'], ngram_range=best_params['tfidf__ngram_range'])
    nb_classifier = MultinomialNB(alpha=best_params['clf__alpha'])
    nb_model = Pipeline([('tfidf', tfidf_vectorizer), ('clf', nb_classifier)])
    nb_model.fit(X_train, y_train)
    return nb_model

# Adım 7: Sonuçları raporlayan fonksiyon
def report_results(model, X_test, y_test):
    predictions = model.predict(X_test)
    report = classification_report(y_test, predictions)
    print(report)

# Ana fonksiyon
def main():
    # Adım 1: Veri setini indirme
    data = download_dataset()

    # Adım 2: Veriyi temizleme
    data['text'] = data['text'].apply(clean_text)

    # Adım 3: Eksik değerleri doldurma
    data = fill_missing_values(data)

    # Adım 4: Veriyi bölme
    X_train, X_test, y_train, y_test = split_data(data)

    # Adım 5: Hiperparametre optimizasyonu
    best_params = optimize_hyperparameters(X_train, y_train)

    # Adım 6: Modeli eğitme
    model = train_model(X_train, y_train, best_params)

    # Adım 7: Sonuçları raporlama
    report_results(model, X_test, y_test)

# Ana fonksiyonu çağırma
if __name__ == "__main__":
    main()



               precision    recall  f1-score   support

     business       0.98      0.96      0.97       103
entertainment       1.00      0.98      0.99        84
     politics       0.98      0.99      0.98        80
        sport       1.00      0.99      0.99        98
         tech       0.95      1.00      0.98        80

     accuracy                           0.98       445
    macro avg       0.98      0.98      0.98       445
 weighted avg       0.98      0.98      0.98       445

